Author: Chris J. Dallmann \
Affiliation: University of Wuerzburg \
Last revision: 11-March-2025

In [1]:
from caveclient import CAVEclient
import numpy as np

In [2]:
# Initialize CAVEclient
# This step requires an authentication token, see https://caveclient.readthedocs.io/en/latest/guide/authentication.html
client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.materialize.version = 840

In [3]:
timestamp = '2024-01-17T08:10:01.179472'

In [4]:
# Specify 9A IDs 
# client.materialize.version = 840
# timestamp = '2024-01-17T08:10:01.179472'
source_ids = [648518346496946148, # chief_9A 
              648518346479879156, 
              648518346479847574, 
              648518346467364359,
              648518346498002535,
              648518346479837078, 
              648518346486716621]

## 9A outputs onto hook 

In [5]:
# Specify target IDs
df_target = client.materialize.query_table('feco_axons_v0')
df_target = df_target.query('cell_type.str.contains("hook")')
target_ids = df_target.pt_root_id.to_list()

In [6]:
syn_thresh = 3
n_syn = np.zeros(len(source_ids))

for index, source_id in enumerate(source_ids):
    df_connectivity = client.materialize.synapse_query(pre_ids=source_id)
    # Remove connections less than syn_thresh  
    df_connectivity['syn_count'] = 1
    df_connectivity = df_connectivity.groupby(['pre_pt_root_id','post_pt_root_id'])['syn_count'].sum().reset_index().sort_values('syn_count', ascending=False)
    df_connectivity = df_connectivity[df_connectivity.syn_count >= syn_thresh]

    # Calculate outputs   
    partner_ids = df_connectivity.post_pt_root_id.to_list()
    for partner_id in partner_ids:
        if partner_id in target_ids:
            n_syn[index] += df_connectivity.syn_count[df_connectivity.post_pt_root_id == partner_id].item() / sum(df_connectivity.syn_count)

# Convert to percent synapses   
n_syn = n_syn*100

In [7]:
n_syn

array([ 62.78350515,   4.19004863,  11.49068323,  54.83870968,
       100.        ,   0.81008101,  17.64705882])

## 9A input from sensory or descending neurons

In [8]:
# Get neuron information
df_neuron_info = client.materialize.query_table('neuron_information')

In [9]:
# Get IDs of target neurons
tag = 'descending' # 'sensory neuron', 'descending'
target_ids = df_neuron_info.pt_root_id[df_neuron_info.tag == tag].tolist()

In [10]:
syn_thresh = 3
n_syn = np.zeros(len(source_ids))

for index, source_id in enumerate(source_ids):
    df_connectivity = client.materialize.synapse_query(post_ids=source_id)
    # Remove connections less than syn_thresh  
    df_connectivity['syn_count'] = 1
    df_connectivity = df_connectivity.groupby(['pre_pt_root_id','post_pt_root_id'])['syn_count'].sum().reset_index().sort_values('syn_count', ascending=False)
    df_connectivity = df_connectivity[df_connectivity.syn_count >= syn_thresh]

    # Calculate inputs   
    partner_ids = df_connectivity.pre_pt_root_id.to_list()
    for partner_id in partner_ids:
        if partner_id in target_ids:
            n_syn[index] += df_connectivity.syn_count[df_connectivity.pre_pt_root_id == partner_id].item() / sum(df_connectivity.syn_count)

# Convert to percent synapses   
n_syn = n_syn*100

In [11]:
n_syn

array([68.31683168,  8.92857143, 12.21374046,  3.7037037 ,  0.        ,
       32.95992034,  0.        ])

## 9A input from premotor neurons

In [16]:
# Specify target IDs
target_df = client.materialize.query_table('left_t1_local_premotor_table_v6')
target_ids = target_df.pt_root_id.to_list()

In [17]:
syn_thresh = 3
n_syn = np.zeros(len(source_ids))

for index, source_id in enumerate(source_ids):
    df_connectivity = client.materialize.synapse_query(post_ids=source_id)
    # Remove connections less than syn_thresh  
    df_connectivity['syn_count'] = 1
    df_connectivity = df_connectivity.groupby(['pre_pt_root_id','post_pt_root_id'])['syn_count'].sum().reset_index().sort_values('syn_count', ascending=False)
    df_connectivity = df_connectivity[df_connectivity.syn_count >= syn_thresh]

    # Calculate inputs   
    partner_ids = df_connectivity.pre_pt_root_id.to_list()
    for partner_id in partner_ids:
        if partner_id in target_ids:
            n_syn[index] += df_connectivity.syn_count[df_connectivity.pre_pt_root_id == partner_id].item() / sum(df_connectivity.syn_count)

# Convert to percent synapses   
n_syn = n_syn*100

In [18]:
n_syn

array([ 6.93069307, 63.83928571, 12.97709924, 53.81263617, 44.56233422,
       35.9970127 , 70.41420118])